In [9]:
import os

main_dir = os.getcwd()
hd_dir = os.path.join(main_dir, 'HD_3.0')
examples_dir = os.path.join(hd_dir, '03_Examples')
case1_dir = os.path.join(examples_dir, 'case1')
export_dir = os.path.join(main_dir, 'results')

In [24]:
def host_export(host_path, export_dir):
    host_file = open(host_path, 'r')
    host_lines = host_file.readlines()
    host_file.close()
    num_atoms = host_lines[1].split()[0]
    
    xyz_lines = ''
    for line in host_lines[2:2+int(num_atoms)]:
        name = line.split()[0]
        x, y, z = line.split()[2], line.split()[3], line.split()[4]
        xyz_lines += name + '\t' + x + '\t' + y + '\t' + z + '\n'
        
    host_name = os.path.basename(host_path)     
    xyz_path = os.path.join(export_dir, host_name + '.xyz')
    xyz_file = open(xyz_path, 'w')
    xyz_file.write(num_atoms + '\n')
    xyz_file.write(host_name + '\n')
    xyz_file.write(xyz_lines)
    xyz_file.close()

In [26]:
hosta_dir = os.path.join(case1_dir, 'hostb')

host_export(hosta_dir, export_dir)